# Using the line and memory profilers
## `line_profiler`

In [1]:
import numpy as np

In [2]:
def euclidean_broadcast(x, y):
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]
    return (diff * diff).sum(axis=2)

In [3]:
def euclidean_trick(x, y):
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]
    xy = np.dot(x, y.T)
    return np.abs(x2 + y2 - 2. * xy)

In [4]:
nsamples = 2000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_broadcast(x, x)
%timeit euclidean_trick(x, x)

1.35 s ± 2.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
84.5 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%load_ext line_profiler

In [6]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
  %lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These stat

In [7]:
%lprun -f euclidean_trick euclidean_trick(x, x)

Timer unit: 1e-06 s

Total time: 0.080869 s
File: <ipython-input-3-ecde47918451>
Function: euclidean_trick at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def euclidean_trick(x, y):
     2         1        141.0    141.0      0.2      x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
     3         1         68.0     68.0      0.1      y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]
     4         1      28972.0  28972.0     35.8      xy = np.dot(x, y.T)
     5         1      51688.0  51688.0     63.9      return np.abs(x2 + y2 - 2. * xy)

In [8]:
%lprun -f euclidean_broadcast euclidean_broadcast(x,x)

Timer unit: 1e-06 s

Total time: 1.3668 s
File: <ipython-input-2-4eb3d9a1042d>
Function: euclidean_broadcast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def euclidean_broadcast(x, y):
     2         1     652249.0 652249.0     47.7      diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]
     3         1     714547.0 714547.0     52.3      return (diff * diff).sum(axis=2)

## cProfiler

In [9]:
%prun euclidean_trick(x, x)

         29 function calls in 0.079 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.052    0.052    0.078    0.078 <ipython-input-3-ecde47918451>:1(euclidean_trick)
        3    0.026    0.009    0.027    0.009 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.001    0.001    0.079    0.079 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.c_einsum}
        1    0.000    0.000    0.079    0.079 {built-in method builtins.exec}
        1    0.000    0.000    0.026    0.026 <__array_function__ internals>:2(dot)
       10    0.000    0.000    0.000    0.000 einsumfunc.py:995(_einsum_dispatcher)
        2    0.000    0.000    0.000    0.000 einsumfunc.py:1004(einsum)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(einsum)
        1    0.000    0.000    0.000    0.000 {method 'disable' 